###  신용 카드 사용 기록 EDA.

#### 0. 필요한 패키지를 불러온다.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
pd.set_option("precision",2)

#### 1. 데이터를 불러와서 살펴 본다.

In [ ]:
df = pd.read_csv('data_credit.csv')

In [ ]:
df.shape

In [ ]:
df.head(5)

#### 2. 데이터를 정제하고 결측치 처리를 한다.

In [ ]:
# 결측치가 분명히 있는 것 같으니 컬럼별 집계해 본다.
nas = df.isnull()
print(nas.sum(axis=0))

In [ ]:
# installments 변수의 도수분포를 본다.
df['installments'].value_counts()

In [ ]:
# 일시불이 NA로 처리 되었으니 결측치를 1로 채운다.
df = df.fillna(value=1)
df.head(5)

In [ ]:
# 다시 installments 변수의 도수분포를 본다.
df['installments'].value_counts()

In [ ]:
# 일단 date와 time은 사용하지 않겠으니 제거한다.
df = df.drop(columns=['date','time'])
df.head(5)

#### 3. 개개 변수를 다양한 방법으로 요약하고 시각화 한다.

3.1. store_id 등장 횟수의 히스토그램.

In [ ]:
freq1 = df['store_id'].value_counts()
freq1.plot.hist(bins=50,color="turquoise",alpha=0.5)
#sns.distplot(freq1.values,bins=50,color="turquoise")      # 다른 방법.
plt.show()

In [ ]:
# 큰 skew를 보이므로 log 변환해 본다.
freq1b = np.log(freq1)
freq1b.plot.hist(bins=30,color="turquoise",alpha=0.5)
plt.show()

In [ ]:
# 중위수를 계산해 본다.
print(np.median(freq1))

In [ ]:
# Top store id를 본다.
freq1.sort_values(ascending=False)[:10]

3.2. card_id 사용 횟수의 히스토그램.

In [ ]:
freq2 = df['card_id'].value_counts()
freq2.plot.hist(bins=50,color="green",alpha=0.5)
#sns.distplot(freq2.values,bins=50,color="green")         # 다른 방법.
plt.show()

In [ ]:
# 큰 skew를 보이므로 log 변환해 본다.
freq2b = np.log(freq2)                                              
freq2b.plot.hist(bins=20,color="green",alpha=0.5)
#sns.distplot(freq2b.values,bins=20,color="green",kde=False)           # 다른 방법.
plt.show()

In [ ]:
# 사용 횟수 Top 10 card id를 본다.
freq2.sort_values(ascending=False)[:10]

3.3. amount의 히스토그램.

In [ ]:
df['amount'].plot.hist(bins=10)
# sns.distplot(df.amount,bins=10,kde=False)
plt.show()

In [ ]:
# 실제 구매 or Cancel.
ser = df.amount > 0
np.round(ser.mean(),4)             # 실제 구매의 퍼센티지.

3.4. installments의 막대그림.

In [ ]:
freq3 = df['installments'].value_counts()
print("installments의 가짓수 = {}".format((freq3.index).nunique()))

In [ ]:
# 도수 분포를 본다.
freq3 = freq3.sort_index()
df2 = pd.DataFrame({"Installment": freq3.index, "Counts":freq3.values})
df2.plot.bar(x="Installment")
#sns.countplot(data=df, x='installments')   # Seaborn의 countplot() 함수로 집계하여 보여준다.  
plt.show()

In [ ]:
# installment = 1 (일시불)이 너무나 많으니 제거하고 다시 시각화 해본다.
freq3b = df[df.installments != 1]['installments'].value_counts()
freq3b = freq3b.sort_index()
df2 = pd.DataFrame({"Installment": freq3b.index, "Counts":freq3b.values})
df2.plot.bar(x="Installment")
#sns.countplot(data=df[df.installments != 1], x='installments')               # 다른 방법.
plt.show()

3.5. days_of_week의 도수 막대그림.

In [ ]:
freq4 = df['days_of_week'].value_counts()
freq4 = freq4.sort_index()
df2 = pd.DataFrame({"Days of Week": freq4.index, "Counts":freq4.values})
df2.plot.bar(x="Days of Week")
print("days of week의 가짓수 = {}".format((freq4.index).nunique()))
plt.show()

In [ ]:
# Seaborn의 countplot() 함수로 집계하여 보여준다.
sns.countplot(data=df, x='days_of_week')
plt.show()

3.6. holiday의 막대그림.

In [ ]:
freq5 = df['holyday'].value_counts()                    # 스펠링 오류!!! 
freq5 = freq5.sort_index()
df2 = pd.DataFrame({"Holiday": freq5.index, "Counts":freq5.values})
df2.plot.bar(x="Holiday")
plt.show()

#### 4. 수치형 변수 + 명목형 변수를 조합한 분석과 시각화.

In [ ]:
# 휴일 여부별 평균 사용 금액.
df.groupby('holyday')['amount'].mean()

In [ ]:
# 요일별 평균 결제 금액.
ser1 = df.groupby('days_of_week')['amount'].mean()
ser1

In [ ]:
# 시각화.
pd.DataFrame({"Days of Week":ser1.index, "Amount":ser1.values}).plot.bar(x="Days of Week")
plt.show()

In [ ]:
# store_id 별 평균 결제 금액.
ser2 = df.groupby('store_id')['amount'].mean()
ser2.plot.hist(x="Store ID",title="Mean Amount by Store",bins=50)
plt.show()

In [ ]:
# store_id 별 Total 결제 금액. Top 10.
ser2b = df.groupby('store_id')['amount'].sum()
ser2b.sort_values(ascending=False)[:10]

In [ ]:
# card_id 별 평균 결제 금액.
ser3 = df.groupby('card_id')['amount'].mean()
ser3.plot.hist(bins=50)
plt.show()

In [ ]:
# card_id 별 Total 결제 금액. Top 10.
ser3b = df.groupby('card_id')['amount'].sum()
ser3b.sort_values(ascending=False)[:10]

#### 5. 등급화.

5.1. store_id 별 Total 결제 금액에 의한 3 등급 분류.

In [ ]:
ser2c = pd.qcut(ser2b,3,labels=['Low', 'Medium', 'High'])                      # qcut() 함수 사용!
combo2 = pd.DataFrame({"Store ID":ser2b.index, "Amount":ser2b.values, "Category":ser2c.values})
combo2.head(10)

In [ ]:
# 등급별 평균.
combo2.groupby("Category")["Amount"].mean()

5.2. card_id 별 Total 결제 금액에 의한 3 등급 분류.

In [ ]:
ser3c = pd.qcut(ser3b,3,labels=['Low', 'Medium', 'High'])
combo3 = pd.DataFrame({"Card ID":ser3b.index,"Amount":ser3b.values, "Category":ser3c.values})
combo3.head(10)

In [ ]:
# 등급별 평균.
combo3.groupby("Category")["Amount"].mean()